In [ ]:
import os 
import sys 
current_dir = os.path.dirname(os.path.abspath('__file__'))
root_dir = os.path.abspath(os.path.join(current_dir, '../../'))
sys.path.append(root_dir)
from __config__ import * 
from torch.utils.data import Dataset, DataLoader 
from glob2 import glob
from scipy.io.wavfile import read as scipy_wavread
import time 
import numpy as np 
import matplotlib.pyplot as plt


In [ ]:
print(WSJ0_MIX_2_8K_PATH)
print(WSJ0_MIX_2_16K_PATH)
# def WSJ0Loader(Dataset):
#     def __init__()

In [ ]:
# check what is happening with the min max folders 

def analyze_wsj0mix(dirpath):
    
    min_dir = os.path.join(dirpath, 'min')
    max_dir = os.path.join(dirpath, 'max')
    
    test_dir_min = os.path.join(min_dir, 'tt')
    test_dir_max = os.path.join(max_dir, 'tt')
#     create the dic for all mixture keys
    
    
    mixtures_dir = os.path.join(test_dir_min, 'mix')
    files = glob(mixtures_dir + '/*.wav')
    unique_ids = set([os.path.basename(f) for f in files])
    
#     check about the files in min and max 
    folders = ['s1', 's2', 'mix']
    for uid in unique_ids: 
        min_files = [(fo, os.path.join(test_dir_min, fo, uid)) for fo in folders]
        min_files = dict([(speaker, scipy_wavread(path)[1]) 
                          for (speaker, path) in min_files])
        
        max_files = [(fo, os.path.join(test_dir_max, fo, uid)) for fo in folders]
        max_files = dict([(speaker, scipy_wavread(path)[1]) 
                          for (speaker, path) in max_files])
        
        max_len = max(max_files['s1'].shape[0], max_files['s2'].shape[0])
        min_len = min(min_files['s1'].shape[0], min_files['s2'].shape[0])
        
        assert(max_files['mix'].shape[0] == max_len)
        assert(min_files['mix'].shape[0] == min_len)
        
        condition = np.allclose(max_files['mix'][:min_len], 
                                min_files['mix'], atol=10e0)
        
#         if not condition:
#             print(max_files['mix'][:min_len].shape)
            
#             print(min_files['mix'].shape)
#             print(min_len)
            
#             time=np.linspace(0, min_len, num=min_len)
#             print(time.shape)
#             plt.figure(3)
#             plt.title('Mixture')
#             plt.plot(time, max_files['mix'][:min_len], 
#                      time, min_files['mix'], 'r', alpha=0.6)
#             plt.show()
            
        condition = np.allclose(min_files['mix'], 
                                min_files['s1']+min_files['s2'], 
                                atol=10e0)
        assert(condition)
        
        if not condition:
            time=np.linspace(0, min_len, num=min_len)
            print(time.shape)
            plt.figure(3)
            plt.title('Mixture')
            plt.plot(time, min_files['s1']+min_files['s2'], 
                     time, min_files['mix'], 'r', alpha=0.3)
            plt.show() 
        
    return len(unique_ids)

before = time.time()
n_files = analyze_wsj0mix(WSJ0_MIX_2_8K_PATH)
now = time.time()
print("Read {} files in {} secs".format(n_files, now - before))

In [ ]:
# Preprocess WSJ0 mix dataset in order to be in universal format
class WSJPytorchDataset(Dataset):
    """Dataset class for time-domian speech separation."""

    def __init__(self,
                 dirpath,
                 sample_clip_size=4):
        """
        Args:
            sample_clip_size: segmental length (default: 4s)
        """
        self.dirpath = dirpath 
        self.fs = self.infer_sampling_rate(self.dirpath)
        print(self.fs)
        
    @staticmethod 
    def infer_sampling_rate(self, dirpath)
        
        print(dirpath)
        
    def __len__(self):
        return len(self.retrieve_index)

    def __getitem__(self, index):
        utt_id, sample_index = self.retrieve_index[index]
        mix_sample = wavread(self.mix_path[utt_id])[0]
        s1_sample = wavread(self.s1_path[utt_id])[0]
        s2_sample = wavread(self.s2_path[utt_id])[0]
        if sample_index == -1:
            length = len(mix_sample)
            stack_length = self.segment_length - length
            mix_stack_sample = mix_sample[: stack_length].reshape(-1, 1)
            s1_stack_sample = s1_sample[: stack_length].reshape(-1, 1)
            s2_stack_sample = s2_sample[: stack_length].reshape(-1, 1)
            mix_clipped_sample = np.concatenate(
                    (mix_sample, mix_stack_sample), axis=0)
            s1_clipped_sample = np.concatenate(
                    (s1_sample, s1_stack_sample), axis=0)
            s2_clipped_sample = np.concatenate(
                    (s2_sample, s2_stack_sample), axis=0)
        else:
            end_index = sample_index + self.segment_length
            mix_clipped_sample = mix_sample[sample_index : end_index]
            s1_clipped_sample = s1_sample[sample_index : end_index]
            s2_clipped_sample = s2_sample[sample_index : end_index]
        src_clipped_sample = np.stack(
            (s1_clipped_sample, s2_clipped_sample), axis=0).squeeze(-1)
        sample = {
            'mix': mix_clipped_sample.reshape(1, -1),
            'src': src_clipped_sample.reshape(2, -1),
        }
        return sample
